# Sistem penjadwalan praktikum lab otomatis menggunakan algoritma meta-heuristik hibrida berbasiskan API. Menggunakan algoritma genetika dan lainnya (dalam pengembangan).

Bikin ulang dari awal, nyoba struktur baru kali aja lebih efisien.

## Setup

In [1]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()
from django.conf import settings
#set debug to true
settings.DEBUG = True

#logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Data related setup

#### Ekstraksi data

In [2]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter
from functools import lru_cache
#Data Handling, import data from model to an object
# All random methods are for testing purposes only

from collections import namedtuple
ModuleDate = namedtuple('ModuleDate', ['start_date', 'end_date'])

class LaboratoryData:

    @classmethod
    @lru_cache(maxsize=1)
    def get_laboratories(cls):
        return Laboratory.objects.all()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_laboratory(cls, id):
        return Laboratory.objects.get(id=id)
    
    @classmethod
    def get_random_laboratory(cls):
        return Laboratory.objects.order_by('?').first()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_assistants(cls, id):
        laboratory = cls.get_laboratory(id)
        if laboratory:
            return laboratory.assistants.all()
        return []
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_modules(cls, id):
        laboratory = cls.get_laboratory(id)
        if laboratory:
            return laboratory.modules.all()
        return []
        

class ModuleData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_modules(cls):
        return Module.objects.all()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_module(cls, id):
        return Module.objects.get(id=id)
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_dates(cls, id):
        module = cls.get_module(id)
        if module:
            return ModuleDate(module.start_date, module.end_date)
        return None

    
    @classmethod
    def get_random_module(cls):
        return Module.objects.order_by('?').first()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_laboratory(cls, id):
        module = cls.get_module(id)
        if module:
            return module.laboratory
        return None
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_groups(cls, id):
        module = cls.get_module(id)
        if module:
            return module.groups.all()
        return []
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_chapters(cls, id):
        module = cls.get_module(id)
        if module:
            return module.chapters.all()
        return []
    
    @classmethod
    def get_participants(cls, id):
        module = cls.get_module(id)
        if module:
            groups = module.groups.all()
            participants = []
            for group in groups:
                group_memberships = group.group_memberships.all()
                for group_membership in group_memberships:
                    participants.append(group_membership.participant)
            return participants
        return []
    
    @classmethod
    def get_assistants(cls, id):
        module = cls.get_module(id)
        if module:
            assistants = []
            assistants_membership = module.assistant_memberships.all()
            for assistant_membership in assistants_membership:
                assistants.append(assistant_membership.assistant)
            return assistants
        return []
        
    
class ChapterData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_chapters(cls):
        return Chapter.objects.all()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_chapter(cls, id):
        return Chapter.objects.get(id=id)
    
    @classmethod
    def get_module(cls, id):
        chapter = cls.get_chapter(id)
        if chapter:
            return chapter.module
        return None
    
    @classmethod
    def get_random_chapter(cls, id):
        chapter = Chapter.objects.get(id=id)
        if chapter:
            return chapter.module
        return None
    
class ParticipantData:
        
    @classmethod
    @lru_cache(maxsize=10)
    def get_participants(cls):
        return Participant.objects.all()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_participant(cls, id):
        return Participant.objects.get(id=id)
    
    @classmethod
    def get_random_participant(cls):
        return Participant.objects.order_by('?').first()
    
    @classmethod
    def get_groups(cls, id):
        participant = Participant.objects.get(id=id)
        if participant:
            groups_membership = participant.group_memberships.all()
            groups = []
            for group_membership in groups_membership:
                groups.append(group_membership.group)
            return groups
        return []
    
    @classmethod
    def get_modules(cls, id):
        participant = Participant.objects.get(id=id)
        if participant:
            groups_membership = participant.group_memberships.all()
            modules = []
            for group_membership in groups_membership:
                modules.append(group_membership.group.module)
            return modules
        return []
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_schedule(cls, id):
        participant = cls.get_participant(id)
        if participant:
            return participant.regular_schedule
        return None
        
class GroupData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_groups(cls):
        return Group.objects.all()
    
    @classmethod
    def get_group(cls, id):
        return Group.objects.get(id=id)
    
    @classmethod
    def get_random_group(cls):
        return Group.objects.order_by('?').first()
    
    @classmethod
    def get_module(cls, id):
        group = cls.get_group(id)
        if group:
            return group.module
        return None
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_participants(cls, id):
        group = cls.get_group(id)
        if group:
            group_memberships = group.group_memberships.all()
            participants = []
            for group_membership in group_memberships:
                participants.append(group_membership.participant)
            return participants
        return []
    
    @classmethod
    def get_assistants(cls, id):
        group = cls.get_group(id)
        if group:
            module = group.module
            assistants = []
            assistants_membership = module.assistant_memberships.all()
            for assistant_membership in assistants_membership:
                assistants.append(assistant_membership.assistant)
            return assistants
        return []
    
    @classmethod
    def get_participant_schedule(cls, id):
        participants = cls.get_participants(id)
        if participants:
            schedule = []
            for participant in participants:
                schedule.append(participant.regular_schedule)
            return schedule
        return None
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_schedule(cls, id):
        participants_schedule = cls.get_participant_schedule(id)
        if participants_schedule:
            days = participants_schedule[0].keys()
            merged_schedule = {day:{} for day in days}
            for day in days:
                for timeslot in participants_schedule[0][day]:
                    is_available = all([participant_schedule[day][timeslot] for participant_schedule in participants_schedule])
                    merged_schedule[day][timeslot] = is_available
            return merged_schedule
        return None
    
class AssistantData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_assistants(cls):
        return Assistant.objects.all()
    
    @classmethod
    def get_assistant(cls, id):
        return Assistant.objects.get(id=id)
    
    @classmethod
    def get_random_assistant(cls):
        # This is for testing purposes only
        return Assistant.objects.order_by('?').first()
    
    @classmethod
    def get_laboratory(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            return assistant.laboratory
        return None
    
    @classmethod
    def get_modules(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            modules = []
            assistant_membership = assistant.assistant_memberships.all()
            for assistant_membership in assistant_membership:
                modules.append(assistant_membership.module)
            return modules
        return []
    
    @classmethod
    def get_groups(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            modules = cls.get_modules(id)
            groups = []
            for module in modules:
                groups.append(module.groups.all())
            return groups
        return []
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_schedule(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            return assistant.regular_schedule
        return None
    
class DataHelper:
    
    @classmethod
    def get_random_laboratory(cls):
        return LaboratoryData.get_random_laboratory()
    
    @classmethod
    def get_random_module(cls):
        return ModuleData.get_random_module()
    
    @classmethod
    def get_random_participant(cls):
        return ParticipantData.get_random_participant()
    
    @classmethod
    def get_random_group(cls):
        return GroupData.get_random_group()
    
    @classmethod
    def get_random_assistant(cls):
        return AssistantData.get_random_assistant()
    
    @classmethod
    def get_random_chapter(cls, id):
        return ChapterData.get_random_chapter(id)
    
    @classmethod
    def get_group_schedule(cls, id):
        return GroupData.get_schedule(id)
    
    @classmethod
    def get_assistant_schedule(cls, id):
        return AssistantData.get_schedule(id)
    
    @classmethod
    def get_available_schedule(cls, id_assistant, id_group):
        assistants_schedule = AssistantData.get_schedule(id_assistant)
        groups_schedule = GroupData.get_schedule(id_group)
        if assistants_schedule and groups_schedule:
            days = assistants_schedule.keys()
            merged_schedule = {day:{} for day in days}
            for day in days:
                for timeslot in assistants_schedule[day]:
                    is_available = assistants_schedule[day][timeslot] & groups_schedule[day][timeslot]
                    merged_schedule[day][timeslot] = is_available
            return merged_schedule

In [4]:
GroupData.get_schedule(2)

{'Friday': {'Shift1': False,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': True,
  'Shift6': False},
 'Monday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Tuesday': {'Shift1': True,
  'Shift2': False,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Saturday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Thursday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Wednesday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False}}

#### Constants

In [3]:
from collections import namedtuple
class Constant:
    # Data
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

    # Simple Data Structures, I put them here for convenience, don't wanna create a new file for them
    TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])
    GeneData = namedtuple("GeneData", ["laboratory", "module", "chapter", "group", "assistant", "time_slot"])

## Algorithm

### Genetic Algorithm

#### Gene

Initialization

In [4]:
class Gene:
    def __init__(self, laboratory: int, module: int, chapter: int, group: int):
        # Make sure that the data is immutable
        self._laboratory = laboratory
        self._module = module
        self._chapter = chapter
        self._group = group

    @property
    def laboratory(self):
        return self._laboratory
    
    @property
    def module(self):
        return self._module
    
    @property
    def chapter(self):
        return self._chapter
    
    @property
    def group(self):
        return self._group

    def __str__(self):
        return f"Gene(laboratory={self._laboratory}, module={self._module}, chapter={self._chapter}, group={self._group})"
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other: "Gene"):
        return self._laboratory == other.laboratory and self._module == other.module and self._chapter == other.chapter and self._group == other.group
    
    def __deepcopy__(self, memo):
        return self
    
    def __hash__(self):
        return hash((self._laboratory, self._module, self._chapter, self._group))
    
    @property
    def laboratory_data(self):
        return LaboratoryData.get_laboratory(self._laboratory)
    
    @property
    def module_data(self):
        return ModuleData.get_module(self._module)
    
    @property
    def chapter_data(self):
        return ChapterData.get_chapter(self._chapter)
    
    @property
    def group_data(self):
        return GroupData.get_group(self._group)
    
    @property
    def group_schedule(self):
        '''Returns the availability of the group'''
        return GroupData.get_schedule(self._group)

In [5]:
#unit test for Gene
import unittest

class TestGene(unittest.TestCase):
        def test_gene_creation(self):
            gene = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
            self.assertEqual(gene.laboratory_data, LaboratoryData.get_laboratory(gene.laboratory))
            self.assertEqual(gene.module_data, ModuleData.get_module(gene.module))
            self.assertEqual(gene.chapter_data, ChapterData.get_chapter(gene.chapter))
            self.assertEqual(gene.group_data, GroupData.get_group(gene.group))
            self.assertEqual(gene.group_schedule, GroupData.get_schedule(gene.group))
            
        def test_gene_equality(self):
            gene1 = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
            gene2 = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
            self.assertEqual(gene1, gene1)
            self.assertNotEqual(gene1, gene2)

        def test_gene_deepcopy(self):
            gene1 = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
            gene2 = gene1.__deepcopy__(None)
            self.assertEqual(gene1, gene2)
            self.assertEqual(gene1.laboratory, gene2.laboratory)
            self.assertEqual(gene1.module, gene2.module)
            self.assertEqual(gene1.chapter, gene2.chapter)
            self.assertEqual(gene1.group, gene2.group)
            
        def test_immutable(self):
            gene = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
            with self.assertRaises(AttributeError):
                gene.laboratory = 1
            with self.assertRaises(AttributeError):
                gene.module = 1
            with self.assertRaises(AttributeError):
                gene.chapter = 1
            with self.assertRaises(AttributeError):
                gene.group = 1
            with self.assertRaises(AttributeError):
                gene.laboratory_data = 1
            with self.assertRaises(AttributeError):
                gene.module_data = 1
            with self.assertRaises(AttributeError):
                gene.chapter_data = 1
            with self.assertRaises(AttributeError):
                gene.group_data = 1
            with self.assertRaises(AttributeError):
                gene.group_schedule = 1
            


In [6]:
# #10000 deepcopies of gene test
# import copy
# def test_deepcopy():
#     gene = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id)
#     for i in range(10000000):
#         gene_copy = gene
#     return gene_copy

# import cProfile
# cp = cProfile.Profile()
# cp.enable()
# test_deepcopy()
# cp.disable()

In [7]:
#Gene Wrapper
class GeneData:
    def __init__(self, gene: Gene, assistant: int, time_slot: Constant.TimeSlot):
        #immutable
        self._laboratory = gene.laboratory
        self._module = gene.module
        self._chapter = gene.chapter
        self._group = gene.group

        #mutable
        self._assistant = assistant
        self._time_slot = time_slot

    @property
    def laboratory(self):
        return self._laboratory
    
    @property
    def module(self):
        return self._module
    
    @property
    def chapter(self):
        return self._chapter
    
    @property
    def group(self):
        return self._group
    
    @property
    def assistant(self):
        return self._assistant
    
    @property
    def time_slot(self):
        return self._time_slot
    
    #setters
    @assistant.setter
    def assistant(self, assistant):
        self._assistant = assistant

    @time_slot.setter
    def time_slot(self, time_slot):
        self._time_slot = time_slot

    def __str__(self):
        return f"GeneData(laboratory={self._laboratory}, module={self._module}, chapter={self._chapter}, group={self._group}, assistant={self._assistant}, time_slot={self._time_slot})"
    
    def __repr__(self):
        return self.__str__()


#### Chromosome

Initialization

In [8]:
# Chromosome class
from typing import List
import copy
class Chromosome:
    def __init__(self, genes: list = []):
        # # Parallel list of genes and gene_data and time_slot, length of both list must be the same.
        # self._genes_list = genes
        #self._gene_data_list = [{"gene": gene, "assistant": assistant, "time_slot": time_slot} for gene, assistant, time_slot in zip(genes, assistant_data, time_slot_data)] 
        #self._gene_data_list = [GeneData(gene, assistant, time_slot) for gene, assistant, time_slot in zip(genes, assistant_data, time_slot_data)] if genes else []
        self._gene_data_list = [{"laboratory": gene["laboratory"], "module": gene["module"], "chapter": gene["chapter"], "group": gene["group"], "assistant": gene["assistant"], "time_slot": gene["time_slot"]} for gene in genes] if genes else []
        self.fitness = 0

    # @property
    # def genes(self):
    #     return self._genes_list
    
    @property
    def gene_data(self):
        return self._gene_data_list
    
    def __str__(self):
        return f"Chromosome(length={len(self._gene_data_list)}, fitness={self.fitness})"
    
    def __repr__(self):
        return self.__str__()
    
    def __getitem__(self, index):
        return self._gene_data_list[index]

    def __len__(self):
        return len(self._gene_data_list)
    
    def __eq__(self, other: "Chromosome"):
        return self._gene_data_list == other.gene_data
    
    def __iter__(self):
        # return as GeneData
        # return iter(Constant.GeneData(gene["gene"].laboratory,
        #                         gene["gene"].module,
        #                         gene["gene"].chapter,
        #                         gene["gene"].group,
        #                         gene["assistant"], 
        #                         gene["time_slot"]
        #                         ) for gene in self._gene_data_list)
        return iter(self._gene_data_list)
    
    # def to_gene_data(self, gene):
    #     return Constant.GeneData(gene["gene"].laboratory,
    #                             gene["gene"].module,
    #                             gene["gene"].chapter,
    #                             gene["gene"].group,
    #                             gene["assistant"], 
    #                             gene["time_slot"]
    #                             )
    
    def transform_to_gene_data(self):
        return [self.to_gene_data(gene) for gene in self._gene_data_list]
    
    def __deepcopy__(self, memo):
        if id(self) in memo:
            return memo[id(self)]
        new_chromosome = Chromosome([])
        #new_chromosome._genes_list = copy.copy(self._genes_list) # Set the reference to the same list, since Gene is immutable.
        #new_chromosome._gene_data_list = [copy.copy(gene_data) for gene_data in self._gene_data_list]
        new_chromosome._gene_data_list = [copy.copy(gene_data) for gene_data in self._gene_data_list]
        new_chromosome.fitness = self.fitness
        #new_chromosome._gene_data_list = self.copy_list(self._gene_data_list)
        memo[id(self)] = new_chromosome
        return new_chromosome
    
    # def copy_list(self, list):
    #     return [copy.copy(item) for item in list]
    
    def __hash__(self):
        return hash(tuple(self._gene_data_list))
    
    def copy(self):
        new_chromosome = Chromosome([])
        #new_chromosome._gene_data_list = [copy.copy(gene_data) for gene_data in self._gene_data_list]
        new_chromosome._gene_data_list = [gene_data.copy() for gene_data in self._gene_data_list]
        new_chromosome.fitness = self.fitness
        return new_chromosome
    
    # def add_gene(self, gene: Gene, assistant: int = None, time_slot: Constant.TimeSlot = None):
    #     #self._gene_data_list.append({"gene": gene, "assistant": assistant, "time_slot": time_slot})
    #     #self._gene_data_list.append(GeneData(gene, assistant, time_slot))
    #     self._gene_data_list.append({"laboratory": gene.laboratory, "module": gene.module, "chapter": gene.chapter, "group": gene.group, "assistant": assistant, "time_slot": time_slot})

    def add_gene(self, laboratory: int, module: int, chapter: int, group: int, assistant: int = None, time_slot: Constant.TimeSlot = None):
        self._gene_data_list.append({"laboratory": laboratory, "module": module, "chapter": chapter, "group": group, "assistant": assistant, "time_slot": time_slot})

    # def remove_gene(self, index):
    #     self._genes_list.pop(index)
    #     self._gene_data_list.pop(index)

    def get_gene(self, index):
        return self._gene_data_list[index]
    
    def get_genes(self):
        return self._gene_data_list

    def set_assistant(self, index, assistant):
        self._gene_data_list[index]["assistant"] = assistant

    def set_time_slot(self, index, time_slot):
        self._gene_data_list[index]["time_slot"] = time_slot

#### Constraints Checker (Gene Level)
Check if a gene is valid by not violating any constraints. Can be used to check if a gene is valid before adding it to a chromosome, or to use it on repair function.

In [9]:
class BaseConstraint:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Constraint(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, gene: Constant.GeneData):
        raise NotImplementedError("Constraint function not implemented")
    
# Ensure that each chapter is assigned to the correct module.
class ChapterModuleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ChapterModuleConstraint")
        self.chapters = ChapterData

    def __call__(self, gene: Constant.GeneData):
        if gene.module == self.chapters.get_module(gene.chapter).id:
            return True
        return False
    
# Ensure that each group is assigned to the correct module.
class GroupModuleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("GroupModuleConstraint")
        self.groups = GroupData
    
    def __call__(self, gene: Constant.GeneData):
        if gene.module == self.groups.get_module(gene.group).id:
            return True
        return False
    
# Ensure that each module is assigned to the correct lab.
class ModuleLaboratoryConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ModuleLaboratoryConstraint")
        self.modules = ModuleData
    
    def __call__(self, gene: Constant.GeneData):
        if gene.laboratory == self.modules.get_laboratory(gene.module).id:
            return True
        return False
    
# Ensure that each assistant is assigned to the correct lab.
class AssistantLaboratoryConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("AssistantLaboratoryConstraint")
        self.assistants = AssistantData
    
    def __call__(self, gene: Constant.GeneData):
        if gene.laboratory == self.assistants.get_laboratory(gene.assistant).id:
            return True
        return False
    
# Ensure that the group schedule is not violated by the gene time slot.
class ScheduleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ScheduleConstraint")
        self.groups = GroupData
    
    def __call__(self, gene: Constant.GeneData):
        schedule = self.groups.get_schedule(gene.group)
        if schedule:
            return schedule[gene.time_slot.day][gene.time_slot.shift]
        return False
    
# Ensure that the assistant schedule is not violated by the gene time slot.
class AssistantScheduleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("AssistantScheduleConstraint")
        self.assistants = AssistantData
    
    def __call__(self, gene: Constant.GeneData):
        schedule = self.assistants.get_schedule(gene.assistant)
        if schedule:
            return schedule[gene.time_slot.day][gene.time_slot.shift]
        return False
    
# Ensure that the group schedule and assistant schedule is not violated each other
class GroupAssistantScheduleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("GroupAssistantScheduleConstraint")
        self.groups = GroupData
        self.assistants = AssistantData
    
    def __call__(self, gene: Constant.GeneData):
        '''Returns true if the group schedule and assistant schedule is not violated each other'''
        group_schedule = self.groups.get_schedule(gene.group)
        assistant_schedule = self.assistants.get_schedule(gene.assistant)
        if group_schedule and assistant_schedule:
            return group_schedule[gene.time_slot.day][gene.time_slot.shift] and assistant_schedule[gene.time_slot.day][gene.time_slot.shift]
        return False
    
# Dynamic Constraint Class (custom constraint function) for on demand constraint
class DynamicConstraint(BaseConstraint):
    def __init__(self, name, constraint_function):
        super().__init__(name)
        self.constraint_function = constraint_function
    
    def __call__(self, gene: Constant.GeneData):
        return self.constraint_function(gene)
    
# Wrapper class for constraint
from typing import List
class ConstraintManager:
    def __init__(self, constraints: List[BaseConstraint]):
        self.constraints = constraints
    
    def __str__(self):
        return f"Constraint(constraints={self.constraints})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, gene: Constant.GeneData):
        return all([constraint(gene) for constraint in self.constraints])

### Fitness Function
Fitness function is a function that takes a chromosome as input and returns a fitness score as output. The fitness score is a measure of how good the chromosome is. The higher the score, the better the chromosome is, wait, is it? Or should I say the lower the score, the better the chromosome is? I don't know, I'm just keep moving forward without any idea of what I'm actually doing, just hoping that maybe I stumble upon something that works. I'm just a monkey with a keyboard, I don't know what I'm doing.

In [10]:
#Base Fitness Class
class BaseFitness:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Fitness(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Fitness function not implemented")

#### 1. Group Assignment Conflict Penalty
This function is used to calculate the number of group that assigned to the same lab at the same time. The lower the number, the less conflict there is. In other words, this objective is to Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

Bahasa Simplenya, fungsi ini menghitung jumlah konflik yang terjadi pada jadwal, khususnya jadwal grup yang berada di lab yang sama pada waktu yang sama. Jadi misal di jadwal hari selasa shift 1, ada 2 grup yang berada di lab yang sama, maka akan dihitung sebagai 1 konflik. Tapi tergantung batas maksimal konflik yang ditentukan, misal batas maksimal konflik adalah 2, maka jadwal tersebut masih valid. Tapi kalau batas maksimal konflik adalah 1, maka jadwal tersebut tidak valid dan dihitung sebagai 1 konflik.

In [11]:
#GroupAssignmentConflictPenalty
from collections import defaultdict

class GroupAssignmentConflictFitness(BaseFitness):
    """Count the number of groups assigned to a time slot in a lab, and penalize the chromosome if the number of groups exceeds the maximum threshold"""
    def __init__(self):
        super().__init__("GroupAssignmentConflictFitness")
        self.max_threshold = 2 # Maximum number of groups that can be assigned to a single time slot in lab
        self.conflict_penalty = 1 # Penalty for each group that exceeds the maximum threshold
        
        #conflicts[laboratory][module][time_slot] = [groups]
        self._conflicts = self.default_outer_dict()
        # Keeps track of the number of groups assigned to a time slot in a lab, initialized to 0

    def default_inner_dict(self):
        return defaultdict(list)
    
    def default_middle_dict(self):
        return defaultdict(self.default_inner_dict)
    
    def default_outer_dict(self):
        return defaultdict(self.default_middle_dict)

    def __call__(self, chromosome: Chromosome):
        self._conflicts.clear()
        for gene in chromosome:
            #gene = chromosome.to_gene_data(gene)
            #self._conflicts[gene.laboratory][gene.module][gene.time_slot].append(gene.group)
            #self._conflicts[gene["gene"].laboratory][gene["gene"].module][gene["time_slot"]].append(gene["gene"].group)
            self._conflicts[gene["laboratory"]][gene["module"]][gene["time_slot"]].append(gene["group"])
        
        total_penalty = 0
        for laboratory in self._conflicts:
            for module in self._conflicts[laboratory]:
                for time_slot in self._conflicts[laboratory][module]:
                    groups = self._conflicts[laboratory][module][time_slot] #Get all groups that are assigned to the time slot, more specifically, the number of groups in conflict[laboratory][module][time_slot]
                    if len(groups) > self.max_threshold:
                        total_penalty += (len(groups) - self.max_threshold) * self.conflict_penalty
        return total_penalty
    
    def get_conflicts(self):
        return self._conflicts
    
    def configure(self, max_threshold, conflict_penalty):
        """Configure the fitness function
        Args:
            max_threshold (int): Maximum number of groups that can be assigned to a single time slot in lab
            conflict_penalty (int): Penalty for each group that exceeds the maximum threshold"""
        self.max_threshold = max_threshold
        self.conflict_penalty = conflict_penalty
        return self

#### 2. Assistant Distribution Penalty
The purpose of this function is to maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading, and to ensure their brain is not fried.

In [12]:
from collections import namedtuple, defaultdict, Counter

#Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
class AssistantDistributionFitness(BaseFitness):
    def __init__(self):
        super().__init__("AssistantDistributionFitness")
        self.max_group_threshold = 15 # Maximum number of groups that can be assigned to a single assistant
        self.max_shift_threshold = 15 # Maximum number of shifts that can be assigned to a single assistant
        self.group_penalty = 1 # Penalty for each group that exceeds the maximum threshold
        self.shift_penalty = 1 # Penalty for each shift that exceeds the maximum threshold

        #groups[assistant][module] = [groups]
        self._groups = self.default_outer_dict()
        #shifts[assistant][module] = [shifts]
        self._shifts = self.default_outer_dict()

    def default_inner_dict(self):
        return defaultdict(set)
    
    def default_outer_dict(self):
        return defaultdict(self.default_inner_dict)
    


    def __call__(self, chromosome: Chromosome):
        self._groups.clear()
        self._shifts.clear()
        for gene in chromosome:
            #gene = chromosome.to_gene_data(gene)
            #self._groups[gene.assistant][gene.module].add(gene.group)
            #self._shifts[gene.assistant][gene.module].add(gene.time_slot)
            #self._groups[gene["assistant"]][gene["gene"].module].add(gene["gene"].group)
            #self._shifts[gene["assistant"]][gene["gene"].module].add(gene["time_slot"])
            self._groups[gene["assistant"]][gene["module"]].add(gene["group"])
            self._shifts[gene["assistant"]][gene["module"]].add(gene["time_slot"])
        
        total_penalty = 0
        for assistant in self._groups:
            for module in self._groups[assistant]:
                groups = self._groups[assistant][module]
                shifts = self._shifts[assistant][module]
                if len(groups) > self.max_group_threshold:
                    total_penalty += (len(groups) - self.max_group_threshold) * self.group_penalty
                if len(shifts) > self.max_shift_threshold:
                    total_penalty += (len(shifts) - self.max_shift_threshold) * self.shift_penalty
        return total_penalty
    
    def get_groups(self):
        return self._groups
    
    def get_shifts(self):
        return self._shifts
    
    def configure(self, max_group_threshold, max_shift_threshold, group_penalty, shift_penalty):
        """Configure the fitness function
        Args:
            max_group_threshold (int): Maximum number of groups that can be assigned to a single assistant
            max_shift_threshold (int): Maximum number of shifts that can be assigned to a single assistant
            group_penalty (int): Penalty for each group that exceeds the maximum threshold
            shift_penalty (int): Penalty for each shift that exceeds the maximum threshold"""
        self.max_group_threshold = max_group_threshold
        self.max_shift_threshold = max_shift_threshold
        self.group_penalty = group_penalty
        self.shift_penalty = shift_penalty
        return self


#### Wrapper Function

In [13]:
#Fitness Manager
class FitnessManager:
    '''FitnessManager is a class that manages all fitness functions and their respective fitness values'''
    def __init__(self, fitness_functions: List[BaseFitness]):
        self.fitness_functions = fitness_functions
    
    def __str__(self):
        return f"FitnessManager(fitness_functions={self.fitness_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome) -> int:
        '''Return the total fitness value of the chromosome'''
        return sum([fitness_function(chromosome) for fitness_function in self.fitness_functions])
    
    def configure(self, fitness_functions: List[BaseFitness]):
        """Configure the fitness manager
        Args:
            fitness_functions (List[BaseFitness]): List of fitness functions"""
        
        self.fitness_functions = fitness_functions

    def grouped_fitness(self, chromosome: Chromosome):
        """Return a dictionary of fitness functions and their respective fitness value"""
        return {fitness_function.name: fitness_function(chromosome) for fitness_function in self.fitness_functions}

### Population

Initialization

In [14]:
#Population Initialization Class
import random

class Population:
    def __init__(self, chromosomes: List[Chromosome], fitness_manager: FitnessManager):
        self.chromosomes = chromosomes
        self.fitness_manager = fitness_manager
    
    def __str__(self):
        return f"Population(chromosomes={self.chromosomes})"
    
    def __repr__(self):
        return self.__str__()
    
    def __getitem__(self, index):
        return self.chromosomes[index]
    
    def __len__(self):
        return len(self.chromosomes)
    
    def __iter__(self):
        return iter(self.chromosomes)
    
    def __eq__(self, other: "Population"):
        return self.chromosomes == other.chromosomes
    
    def __contains__(self, chromosome: Chromosome):
        return chromosome in self.chromosomes
    
    def sort_best(self):
        self.chromosomes.sort(key=lambda chromosome: chromosome.fitness)

    def sort_worst(self):
        self.chromosomes.sort(key=lambda chromosome: chromosome.fitness, reverse=True)
    
    def calculate_fitness(self):
        for chromosome in self.chromosomes:
            chromosome.fitness = self.fitness_manager(chromosome)
    
    def add_chromosome(self, chromosome: Chromosome):
        #if chromosome is list, add all chromosomes in the list
        if isinstance(chromosome, list):
            self.chromosomes.extend(chromosome)
        else:
            self.chromosomes.append(chromosome)

    def remove_chromosome(self, chromosome: Chromosome):
        self.chromosomes.remove(chromosome)

    def pop(self):
        return self.chromosomes.pop()
    
    def get_chromosome(self, index):
        return self.chromosomes[index]
    
    def get_chromosomes(self):
        return self.chromosomes
    
    def get_random_chromosome(self):
        return random.choice(self.chromosomes)
    
    def get_best_chromosome(self):
        return min(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def get_worst_chromosome(self):
        return max(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def get_average_fitness(self):
        return sum([chromosome.fitness for chromosome in self.chromosomes]) / len(self.chromosomes)

### Factory Method
Class for generating population, chromosome, and gene. This class also generates some data like time slot, chromosome length, etc.This class requires a lot of data to be passed to it, so I think it's better to make it a class instead of a function.

In [15]:
#factory.py
from math import ceil, floor
import random
from datetime import timedelta

TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Factory:
    '''Factory class to generate chromosomes and population. It also contains the data from the database. We can call this class when we need some of'''
    def __init__(self):
        self.laboratories = LaboratoryData
        self.modules = ModuleData
        self.chapters = ChapterData
        self.groups = GroupData
        self.participants = ParticipantData
        self.assistants = AssistantData
        self.constant = Constant
        
        #self.constraints = ConstraintManager([ChapterModuleConstraint(), GroupModuleConstraint(), ModuleLaboratoryConstraint(), AssistantLaboratoryConstraint(), ScheduleConstraint()])
        self.fitness_manager = FitnessManager([GroupAssignmentConflictFitness(), AssistantDistributionFitness()])

    # def set_constraints(self, constraints: List[BaseConstraint]):
    #     # No idea if I need to check constraints here when generating the population
    #     # But I think it's better to set the constraints here, maybe we can change the constraints later
    #     self.constraints = ConstraintManager(constraints)

    def set_fitness_manager(self, fitness_manager: FitnessManager):
        self.fitness_manager = fitness_manager

    def set_data(self, laboratories, modules, chapters, groups, participants, assistants):
        self.laboratories = laboratories
        self.modules = modules
        self.chapters = chapters
        self.groups = groups
        self.participants = participants
        self.assistants = assistants
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7) # Number of weeks between start date and end date, using floor to make sure that the time slot does not exceed the end date
        random_weeks = random.randint(0, weeks_duration)
        random_days = random.choice(self.constant.days)
        random_shifts = random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + self.constant.days.index(random_days))
        # random_date = random_date.strftime("%d-%m-%Y")
        return TimeSlot(random_date, random_days, random_shifts)
    
    def generate_time_slot_weekly(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        random_days = random.choice(self.constant.days)
        random_shifts = random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days= 7 + self.constant.days.index(random_days))
        # random_date = random_date.strftime("%d-%m-%Y")
        return TimeSlot(random_date, random_days, random_shifts)
    
    def chromosome_size(self) -> int:
        """
        Calculate the chromosome size based on the number of modules chapters and groups.
        Each group must be assigned to all chapters in a module. Mostly used for testing purposes.
        The actual chromosome size is generated based on parential data.

        Returns:
            int: chromosome size
        """
        chromosome_size = 0
        for module in self.modules:
            chromosome_size += len(module.groups.all()) * len(module.chapters.all())
        return chromosome_size
    
    def generate_chromosome(self) -> Chromosome:
        """Generate a chromosome based on data, each group must be assigned to all chapters in a module of appropriate lab"""
        chromosome = Chromosome([])
        for module in self.modules.get_modules():
            for group in self.modules.get_groups(module.id):
                for chapter in self.modules.get_chapters(module.id):
                    laboratory = module.laboratory
                    assistant = random.choice(self.laboratories.get_assistants(laboratory.id))
                    time_slot = self.generate_time_slot(module.start_date, module.end_date)
                    #gene = Gene(laboratory.id, module.id, chapter.id, group.id)
                    #chromosome.add_gene(gene=gene, assistant=assistant.id, time_slot=time_slot)
                    chromosome.add_gene(laboratory.id, module.id, chapter.id, group.id, assistant.id, time_slot)
        return chromosome
    
    def generate_chromosome_weekly(self) -> Chromosome:
        """Generate a chromosome based on data, each group must be assigned to all chapters in a module of appropriate lab"""
        chromosome = Chromosome([])
        for module in self.modules.get_modules():
            for group in self.modules.get_groups(module.id):
                start_date = module.start_date
                end_date = module.end_date
                duration = (end_date - start_date).days + 1
                weeks_duration = floor(duration / 7)
                chapters_count = len(self.modules.get_chapters(module.id))
                weekly_chapters = ceil(chapters_count / weeks_duration)
                for i in range(weekly_chapters):
                    laboratory = module.laboratory
                    assistant = random.choice(self.laboratories.get_assistants(laboratory.id))
                    time_slot = self.generate_time_slot_weekly(start_date, end_date)
                    #first chapter
                    chapter = module.chapters.all().first()
                    gene = Gene(laboratory.id, module.id, chapter.id, group.id)
                    chromosome.add_gene(gene=gene, assistant=assistant.id, time_slot=time_slot)
        return chromosome
    
    def generate_population(self, population_size: int, fitness_manager: FitnessManager = None, weekly = False) -> Population:
        """Generate a population based on the population size"""

        if settings.DEBUG:
            logger.info("Generating population")

        if fitness_manager:
            self.fitness_manager = fitness_manager

        if weekly:
            return Population([self.generate_chromosome_weekly() for _ in range(population_size)], self.fitness_manager)
        else:
            return Population([self.generate_chromosome() for _ in range(population_size)], self.fitness_manager)
            
        # chromosomes = [generator() for _ in range(population_size)]
        # population = Population(chromosomes, self.fitness_manager)
        # population.calculate_fitness()
        # return population
    
    # def generate_population_weekly(self, population_size: int) -> Population:
    #     """Generate a population based on the population size"""

    #     if settings.DEBUG:
    #         logger.info("Generating population")

    #     chromosomes = []
    #     # chromosomes = [self.generate_chromosome() for i in range(population_size)]
    #     for i in range(population_size):
    #         if settings.DEBUG:
    #             logger.info(f"Generating chromosome {i}")

    #         chromosomes.append(self.generate_chromosome_weekly())

    #     population = Population(chromosomes, self.fitness_manager)
    #     #population.calculate_fitness()
    #     return population
    


In [16]:
factory = Factory()
population = factory.generate_population(100)

INFO:__main__:Generating population


### Operator
Operator is a class that contains methods for performing genetic operations such as crossover and mutation. This class is used by the genetic algorithm class to perform genetic operations on the population.

#### Mutation

The one that need to mutate is just the time slot and the assistant, the rest is fixed because we just need the schedule. 
The group is already assigned to the appropriate module and all its chapters, we don't need to change it. 
Each chapter also already assigned to the correct module, and each module have their own lab, as well the group and the assistant. 
The schedule is affected by the time slot and the assistant that is assigned to the group. 
The number of group is fixed and cannot be random, that's why the only things that need to be mutated is the time slot and the assistant only.

Tapi masih rada bingung sih fungsi mutasi apa aja yang mesti diterapin, apa ini bener?

In [17]:
#Mutation Class

import random
from copy import deepcopy

class BaseMutation:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the mutation function being called if more than one mutation function is used.
    
    def __str__(self):
        return f"Mutation(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Mutation function not implemented")
    
class SwapMutation(BaseMutation):
    def __init__(self):
        super().__init__("SwapMutation")
    
    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene1 = random.choice(chromosome)
        # Randomly select another gene
        gene2 = random.choice(chromosome)
        # Swap the time slot and the assistant
        gene1['assistant'], gene2['assistant'] = gene2['assistant'], gene1['assistant']
        gene1['time_slot'], gene2['time_slot'] = gene2['time_slot'], gene1['time_slot']
        #gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
        #gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
        return chromosome
    
class ShiftMutation(BaseMutation):
    def __init__(self):
        super().__init__("ShiftMutation")
        self.constant = Constant
    
    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene = random.choice(chromosome)
        # Shift the time slot
        gene['time_slot'] = self.shift_time_slot(gene['time_slot'])
        #gene.time_slot = self.shift_time_slot(gene.time_slot)

        return chromosome
    
    def shift_time_slot(self, time_slot: TimeSlot) -> TimeSlot:
        # Shift the time slot by 1 day
        if time_slot.day == self.constant.days[-1]:
            return TimeSlot(time_slot.date + timedelta(days=2), self.constant.days[0], time_slot.shift)
        return TimeSlot(time_slot.date + timedelta(days=1), self.constant.days[self.constant.days.index(time_slot.day) + 1], time_slot.shift)
    
class RandomMutation(BaseMutation):
    def __init__(self):
        super().__init__("RandomMutation")
        self.constant = Constant
        self.laboratories = LaboratoryData
        self.modules = ModuleData

    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene_data = random.choice(chromosome)
        #module = gene_data['gene'].module_data
        module_date = self.modules.get_dates(gene_data['module'])
        # Randomly select a time slot
        time_slot = self.generate_time_slot(module_date.start_date, module_date.end_date)
        # Randomly select an assistant
        #assistant = random.choice(self.laboratories.get_assistants(gene_data['gene'].laboratory)).id
        assistant = random.choice(self.laboratories.get_assistants(gene_data['laboratory'])).id
        # Change the gene
        gene_data['time_slot'] = time_slot
        #gene_data.time_slot = time_slot
        gene_data['assistant'] = assistant
        #gene_data.assistant = assistant
        return chromosome
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7)
        random_weeks = random.randint(0, weeks_duration)
        random_days = random.choice(self.constant.days)
        random_shifts = random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + self.constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)
    
class DynamicMutation(BaseMutation):
    def __init__(self, name, mutation_function):
        super().__init__(name)
        self.mutation_function = mutation_function
    
    def __call__(self, chromosome: Chromosome):

        return self.mutation_function(chromosome)
    
class MutationManager:
    def __init__(self, mutation_functions: List[BaseMutation]):
        self.mutation_functions = mutation_functions
        self.mutation_probability = 0.1
    
    def __str__(self):
        return f"MutationManager(mutation_functions={self.mutation_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        #random based on probability weight
        if random.random() < self.mutation_probability:
            if settings.DEBUG:
                logger.info("Mutating chromosome")
            mutation_function = self.get_random_mutation()
            return mutation_function(chromosome)
        return chromosome
    
    def get_random_mutation(self):
        return random.choices(self.mutation_functions, weights=[mutation.probability_weight for mutation in self.mutation_functions])[0]
    
    def configure(self, mutation_functions: List[BaseMutation]):
        self.mutation_functions = mutation_functions

#### Crossover

In [18]:
#Crossover Class

import random
from copy import deepcopy

class BaseCrossover:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the crossover function being called if more than one crossover function is used.
    
    def __str__(self):
        return f"Crossover(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        raise NotImplementedError("Crossover function not implemented")
    
class SinglePointCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("SinglePointCrossover")
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # Randomly select a point
        point = random.randint(0, len(parent1))
        # initialize the children
        # child1 = deepcopy(parent1)
        # child2 = deepcopy(parent2)
        child1 = parent1 # already deepcopy in the selection part of the algorithm
        child2 = parent2
        #swap that point onwards
        child1.gene_data[point:], child2.gene_data[point:] = child2.gene_data[point:], child1.gene_data[point:]

        return child1, child2
    
class TwoPointCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("TwoPointCrossover")
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # Randomly select 2 points
        point1 = random.randint(0, len(parent1))
        point2 = random.randint(0, len(parent1))
        # initialize the children
        child1 = parent1
        child2 = parent2
        #swap a section of the chromosome between the 2 points
        if point1 > point2:
            point1, point2 = point2, point1
        child1.gene_data[point1:point2], child2.gene_data[point1:point2] = child2.gene_data[point1:point2], child1.gene_data[point1:point2]

        return child1, child2
    
class UniformCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("UniformCrossover")
        self.uniform_probability = 0.5
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # initialize the children
        child1 = parent1
        child2 = parent2
        #swap a section of the chromosome between the 2 points
        count = 0
        for i in range(len(child1)):
            if random.random() < self.uniform_probability:
                count += 1
                child1.gene_data[i], child2.gene_data[i] = child2.gene_data[i], child1.gene_data[i]
        
        return child1, child2
    
    def configure(self, uniform_probability):
        '''Configure the crossover function
        
        Args:
            uniform_probability (float): The probability of swapping a gene between the 2 parents on a particular index'''
        
        self.uniform_probability = uniform_probability

class DynamicCrossover(BaseCrossover):
    def __init__(self, name, crossover_function):
        super().__init__(name)
        self.crossover_function = crossover_function
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        return self.crossover_function(parent1, parent2)
    
class CrossoverManager:
    def __init__(self, crossover_functions: List[BaseCrossover]):
        self.crossover_functions = crossover_functions
        self.crossover_probability = 0.1
    
    def __str__(self):
        return f"CrossoverManager(crossover_functions={self.crossover_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        #random based on probability weight
        if random.random() < self.crossover_probability:
            if settings.DEBUG:
                logger.info("Crossovering chromosome")
            crossover_function = self.get_random_crossover()
            return crossover_function(parent1, parent2)
        return parent1, parent2
    
    def get_random_crossover(self):
        return random.choices(self.crossover_functions, weights=[crossover.probability_weight for crossover in self.crossover_functions])[0]
    
    def configure(self, crossover_functions: List[BaseCrossover]):
        self.crossover_functions = crossover_functions

In [19]:
# # Crossover Test
# settings.DEBUG = True
# factory = Factory()
# population = factory.generate_population(50)

# single_point_crossover = SinglePointCrossover()
# two_point_crossover = TwoPointCrossover()
# uniform_crossover = UniformCrossover()

# crossover_manager = CrossoverManager([single_point_crossover, two_point_crossover, uniform_crossover])
# crossover_manager.crossover_probability = 1

# child1, child2 = single_point_crossover(copy.deepcopy(population[0]), copy.deepcopy(population[1]))

In [20]:
# # count the number of genes that are different
# count = 0
# for i in range(len(child1)):
#     if child1[i] != population[0][i]:
#         count += 1

# print(f"Number of genes that are different: {count}")

#### Repair

In [21]:
#Repairs Class

class BaseRepair:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Repair(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Repair function not implemented")
    
class DynamicRepair(BaseRepair):
    def __init__(self, name, repair_function):
        super().__init__(name)
        self.repair_function = repair_function
    
    def __call__(self, chromosome: Chromosome):
        return self.repair_function(chromosome)
    
class RepairManager:
    def __init__(self, repair_functions: List[BaseRepair]):
        self.repair_functions = repair_functions
    
    def __str__(self):
        return f"RepairManager(repair_functions={self.repair_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        for repair_function in self.repair_functions:
            chromosome = repair_function(chromosome)
        return chromosome
    
    def configure(self, repair_functions: List[BaseRepair]):
        self.repair_functions = repair_functions

In [22]:
class RepairTimeSlot(BaseRepair):
    def __init__(self):
        super().__init__("RepairTimeSlot")
        self.schedule_constraint = ScheduleConstraint()
        self.module_data = ModuleData
        self.group_data = GroupData
    
    def __call__(self, chromosome: Chromosome):
        # chromosome = deepcopy(chromosome)
        for index, gene in enumerate(chromosome):
            #gene = chromosome.to_gene_data(gene)
            #gene = Constant.GeneData(gene['gene'].laboratory, gene['gene'].module, gene['gene'].chapter, gene['gene'].group, gene['assistant'], gene['time_slot'])
            start_date = self.module_data.get_dates(gene['module']).start_date
            end_date = self.module_data.get_dates(gene['module']).end_date
            schedule = self.group_data.get_schedule(gene['group'])
            if not self.check_available_time_slot(gene['time_slot'], schedule):
                time_slot = self.find_feasible_solution(start_date, end_date, schedule)
                if time_slot is None:
                    time_slot = gene['time_slot']
                chromosome.set_time_slot(index, time_slot)
        return chromosome
    
    def check_available_time_slot(self,time_slot: TimeSlot, schedule=None):
        #schedule = self.group_data.get_schedule(group)
        if schedule:
            return schedule[time_slot.day][time_slot.shift]
        return False
    
    def find_feasible_solution(self, start_date, end_date, schedule, max_iteration=100):
        """Find a feasible solution for the gene by randomly generating a time slot until it is available"""
        for _ in range(max_iteration):
            time_slot = self.choose_available_time_slot(start_date, end_date, schedule)
            if self.check_available_time_slot(time_slot, schedule):
                return time_slot
        return None
    
    def choose_available_time_slot(self,start_date, end_date, schedule):
        """Choose a random available time slot for the gene"""
        #start_date = self.module_data.get_module(gene.module).start_date
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        #end_date = self.module_data.get_module(gene.module).end_date
        week_duration = (end_date - start_date).days + 1
        #get the group schedule
        #available_time_slots = self.group_data.get_schedule(group)
        #get all available time slots, [day, shift]
        available_time_slots = [TimeSlot(start_date, day, shift) for day, shifts in schedule.items() for shift, available in shifts.items() if available]
        #if there is no available time slot, then return a random time slot
        if len(available_time_slots) == 0:
            return self.generate_time_slot(start_date, end_date)
        
        random_time_slot = random.choice(available_time_slots)
        random_week = random.randint(0, week_duration)
        #calculate the date
        random_date = start_date + timedelta(days=random_week * 7 + Constant.days.index(random_time_slot.day))
        return TimeSlot(random_date, random_time_slot.day, random_time_slot.shift)
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7)
        random_weeks = random.randint(0, weeks_duration)
        random_days = random.choice(Constant.days)
        random_shifts = random.choice(Constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + Constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)

In [23]:
#test def generate_time_slot
factory = Factory()
repair_time_slot = RepairTimeSlot()
population = factory.generate_population(20)

def test_repair_time_slot():
    for chromosome in population:
        chromosome = repair_time_slot(chromosome)

INFO:__main__:Generating population


In [24]:
# settings.DEBUG = False

# factory = Factory()
# population = factory.generate_population(50)

# repair_time_slot = RepairTimeSlot()

# repair_manager = RepairManager([repair_time_slot])

# schedule_constraint = ScheduleConstraint()

# for chromosome in population:
#     chromosome = repair_manager(chromosome)
#     count = 0
#     for gene in chromosome:
#         if not schedule_constraint(gene):
#             count += 1
#     print(f"Number of genes that violate the schedule constraint: {count}")

#### Selection

In [25]:
#Selection Class

import random

class BaseSelection:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the selection function being called if more than one selection function is used.
    
    def __str__(self):
        return f"Selection(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, population: Population):
        raise NotImplementedError("Selection function not implemented")
        
    
class RouletteWheelSelection(BaseSelection):
    def __init__(self):
        super().__init__("RouletteWheelSelection")
    
    def __call__(self, population: Population):
        # Calculate the total fitness
        total_fitness = sum([chromosome.fitness for chromosome in population])
        # Calculate the probability of each chromosome being selected
        probabilities = [chromosome.fitness / total_fitness for chromosome in population]
        # Select a chromosome based on the probability

        return random.choices(population, weights=probabilities)[0] # random.choices return a list, so we need to get the first element
    
class TournamentSelection(BaseSelection):
    def __init__(self):
        super().__init__("TournamentSelection")
        self.tournament_size = 2
    
    def __call__(self, population: Population):
        # Select a random chromosome
        chromosome = random.choice(population)
        # Select a random chromosome from the tournament size
        for i in range(self.tournament_size - 1):
            chromosome2 = random.choice(population)
            if chromosome2.fitness < chromosome.fitness:
                chromosome = chromosome2 # Select the chromosome with the lowest fitness

        return chromosome
    
    def configure(self, tournament_size):
        self.tournament_size = tournament_size

class ElitismSelection(BaseSelection):
    def __init__(self):
        super().__init__("ElitismSelection")
        self.elitism_size = 1
    
    def __call__(self, population: Population):
        # Sort the population based on fitness
        population = sorted(population, key=lambda chromosome: chromosome.fitness)
        # Select the best chromosome

        if self.elitism_size == 1:
            return population[0]
        
        return population[:self.elitism_size]
    
    def configure(self, elitism_size):
        self.elitism_size = elitism_size

class DynamicSelection(BaseSelection):
    def __init__(self, name, selection_function):
        super().__init__(name)
        self.selection_function = selection_function
    
    def __call__(self, population: Population):
        return self.selection_function(population)
    
class SelectionManager:
    def __init__(self, selection_functions: List[BaseSelection]):
        self.selection_functions = selection_functions
        self.selection_probability = 0.1
    
    def __str__(self):
        return f"SelectionManager(selection_functions={self.selection_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, population: Population) -> Chromosome:
        #random based on probability weight
        if random.random() < self.selection_probability:
            if settings.DEBUG:
                logger.info("Selecting chromosome")
            selection_function = self.get_random_selection()
            return selection_function(population)
        
        # if isinstance(population, list):
        #     return random.choice(population)

        return population.get_random_chromosome()
    
    def get_random_selection(self):
        return random.choices(self.selection_functions, weights=[selection.probability_weight for selection in self.selection_functions])[0]
    
    def configure(self, selection_functions: List[BaseSelection]):
        self.selection_functions = selection_functions

### Main Algorithm

In [26]:
#Genetic Algorithm Class

from copy import deepcopy

class GeneticAlgorithm:
    def __init__(self):
        self.factory = Factory()
        self.population_size = 10
        self.fitness_manager = FitnessManager([GroupAssignmentConflictFitness(), AssistantDistributionFitness()])
        self.selection_manager = SelectionManager([RouletteWheelSelection(), TournamentSelection(), ElitismSelection()])
        self.crossover_manager = CrossoverManager([SinglePointCrossover(), TwoPointCrossover(), UniformCrossover()])
        self.mutation_manager = MutationManager([SwapMutation(), ShiftMutation(), RandomMutation()])
        self.repair_manager = RepairManager([RepairTimeSlot()])
        self.elitism_size = 1
        self.elitism_selection = ElitismSelection()

        self.initial_solution = None

        self.log = {}

    def __str__(self):
        return f"GeneticAlgorithm(factory={self.factory}, population_size={self.population_size}, selection_manager={self.selection_manager}, crossover_manager={self.crossover_manager}, mutation_manager={self.mutation_manager}, repair_manager={self.repair_manager}, elitism_size={self.elitism_size})"
    
    def __repr__(self):
        return self.__str__()
    
    def _init_population(self, weekly = False): #Protected method, can only be called from the class itself.
        #return self.factory.generate_population_weekly(self.population_size)
        
        self.elitism_selection.elitism_size = self.elitism_size
        return self.factory.generate_population(self.population_size, self.fitness_manager)
    
    def __selection(self, population: Population): #Private method, can only be called from the class itself, subclass can't call this method.
        return self.selection_manager(population).copy()
    
    def __elitism(self, population: Population):
        return self.elitism_selection(population).copy()
    
    def __crossover(self, parent1: Chromosome, parent2: Chromosome): #Private method, can only be called from the class itself, subclass can't call this method.
        return self.crossover_manager(parent1, parent2)
    
    def __mutation(self, chromosome: Chromosome): #Private method, can only be called from the class itself, subclass can't call this method.
        return self.mutation_manager(chromosome)
    
    def __repair(self, chromosome: Chromosome):
        return self.repair_manager(chromosome)
    
    def __evolve(self, population: Population):

        # Selection
        parent1 = self.__selection(population)
        parent2 = self.__selection(population)
        #parent1 = self.__selection(population).copy()
        #parent2 = self.__selection(population).copy()

        # Crossover
        child1, child2 = self.__crossover(parent1, parent2) # Crossover is done using deep copy, so we need to assign it back to the variable since it not modify the original chromosome

        # Mutation
        self.__mutation(child1) # Mutation is done in place, so we don't need to assign it back to the variable
        self.__mutation(child2)

        #Repair
        self.__repair(child1) # Repair is done in place, so we don't need to assign it back to the variable
        self.__repair(child2)
        
        return child1, child2
    
    def __evolve_parallel(self, args):
        population, iteration = args
        child1, child2 = self.__evolve(population, iteration)
        return child1, child2
    
    def _evolve_population(self, population: Population):
    
        elitism = self.__elitism(population)
        children = []

        while len(children) < len(population) - self.elitism_size:
            child1, child2 = self.__evolve(population)
            children.append(child1)
            children.append(child2)
            
        return Population(children, population.fitness_manager), elitism
    
    def log_detail(self, i, best_chromosome, worst_chromosome):
        self.log[i] = {"best_chromosome": best_chromosome, "worst_chromosome": worst_chromosome}

    def run(self, max_iteration: int):
        population = self._init_population()
        population.calculate_fitness()
        population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
        
        for i in range(max_iteration):
            
            population, elitism = self._evolve_population(population)
            population.add_chromosome(elitism)
            population.calculate_fitness()

            # Sort the population based on fitness
            population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
            self.log_detail(i, population[0].fitness, population[-1].fitness)
            if population[0].fitness == 0:
                logger.info(f"Found the best solution after {i} iterations, 0 fitness value? Poggers")
                break
        return population[0]
    
    def configure(self, population_size: int = 10, elitism_size: int = 1, factory: Factory = None, fitness_manager: FitnessManager = None, selection_manager: SelectionManager = None, crossover_manager: CrossoverManager = None, mutation_manager: MutationManager = None, repair_manager: RepairManager = None, elitism_selection: ElitismSelection = None):
        '''Configure the genetic algorithm, use None to use the default value.
        '''
        self.population_size = self.population_size if population_size is None else population_size
        self.elitism_size = self.elitism_size if elitism_size is None else elitism_size
        self.elitism_selection = self.elitism_selection if elitism_selection is None else elitism_selection
        self.factory = factory if factory is not None else self.factory
        self.fitness_manager = fitness_manager if fitness_manager is not None else self.fitness_manager
        self.selection_manager = selection_manager if selection_manager is not None else self.selection_manager
        self.crossover_manager = crossover_manager if crossover_manager is not None else self.crossover_manager
        self.mutation_manager = mutation_manager if mutation_manager is not None else self.mutation_manager
        self.repair_manager = repair_manager if repair_manager is not None else self.repair_manager
        
        return self

In [27]:
def generate_best_chromosome(max_iteration, population_size):
    factory = Factory()

    #fitness
    group_assignment_conflict_fitness = GroupAssignmentConflictFitness().configure(3, 1)

    assistant_distribution_fitness = AssistantDistributionFitness().configure(15, 50, 1, 1)

    fitness_manager = FitnessManager([group_assignment_conflict_fitness, assistant_distribution_fitness])

    selection_manager = SelectionManager([RouletteWheelSelection(), TournamentSelection(), ElitismSelection()])
    crossover_manager = CrossoverManager([SinglePointCrossover(), TwoPointCrossover(), UniformCrossover()])
    mutation_manager = MutationManager([SwapMutation(), ShiftMutation(), RandomMutation()])
    repair_manager = RepairManager([RepairTimeSlot()])
    elitism_size = 2
    elitism_selection = ElitismSelection()

    selection_manager.selection_probability = 0.75
    crossover_manager.crossover_probability = 0.75
    mutation_manager.mutation_probability = 0.75

    genetic_algorithm = GeneticAlgorithm().configure(population_size = population_size, 
                                                    elitism_size = elitism_size, 
                                                    factory = factory, 
                                                    fitness_manager = fitness_manager, 
                                                    selection_manager = selection_manager, 
                                                    crossover_manager = crossover_manager,
                                                    mutation_manager = mutation_manager, 
                                                    repair_manager = repair_manager, 
                                                    elitism_selection = elitism_selection)

    return genetic_algorithm.run(max_iteration)

def calculate_fitness(chromosome):
    group_assignment_conflict_fitness = GroupAssignmentConflictFitness()
    group_assignment_conflict_fitness.configure(3, 1)

    assistant_distribution_fitness = AssistantDistributionFitness()
    assistant_distribution_fitness.configure(15, 50, 1, 1)

    fitness_manager = FitnessManager([group_assignment_conflict_fitness, assistant_distribution_fitness])

    return fitness_manager(chromosome)

In [28]:
# settings.DEBUG = False
# import cProfile
# cp = cProfile.Profile()
# cp.enable()
# best_chromosome = generate_best_chromosome(50, 25)
# cp.disable()

### Local Search

In [29]:
class BaseSearch:
    def __init__(self, name):
        self.name = name
        self.fitness_manager = FitnessManager([GroupAssignmentConflictFitness(), AssistantDistributionFitness()])
    
    def __str__(self):
        return f"Search(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Search function not implemented")
    
    def configure(self, fitness_manager: FitnessManager):
        raise NotImplementedError("Configure function not implemented")

#### Neighborhood Structure

In [30]:
# Neighborhood Structure, use parallel processing to speed up the search

import joblib
import random
from copy import deepcopy

class BaseNeighborhood:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Neighborhood(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Neighborhood function not implemented")
    
class SwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("SwapNeighborhood")
    
    def __call__(self, chromosome: Chromosome):
        '''Generate a set of neighbor solutions by swapping 2 elements in the chromosome.
        The time complexity is O(n^2) where n is the number of genes in the chromosome.
        The space complexity is O(n^2) where n is the number of genes in the chromosome.
        Hella slow and expensive.'''
        neighbors = []
        for i in range(len(chromosome)):
            for j in range(i + 1, len(chromosome)):
                neighbor = chromosome.copy()
                neighbor[i]['time_slot'], neighbor[j]['time_slot'] = neighbor[j]['time_slot'], neighbor[i]['time_slot']
                if neighbor[i]['module'] == neighbor[j]['module']:
                    neighbor[i]['assistant'], neighbor[j]['assistant'] = neighbor[j]['assistant'], neighbor[i]['assistant']
                neighbors.append(neighbor)
        return neighbors
    
class RandomSwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("RandomSwapNeighborhood")
        self.neighborhood_size = 100
    
    def __call__(self, chromosome: Chromosome):
        '''Generate a set of neighbor solutions by swapping 2 elements in the chromosome.'''
        neighbors = []
        for _ in range(self.neighborhood_size):
            neighbor = chromosome.copy()
            i = random.randint(0, len(chromosome) - 1)
            j = random.randint(0, len(chromosome) - 1)
            neighbor[i]['time_slot'], neighbor[j]['time_slot'] = neighbor[j]['time_slot'], neighbor[i]['time_slot']
            if neighbor[i]['module'] == neighbor[j]['module']:
                neighbor[i]['assistant'], neighbor[j]['assistant'] = neighbor[j]['assistant'], neighbor[i]['assistant']
            neighbors.append(neighbor)
        return neighbors
    
    def configure(self, neighborhood_size = None):
        self.neighborhood_size = neighborhood_size or self.neighborhood_size
        return self
    
class RandomRangeSwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("RandomRangeSwapNeighborhood")
        self.neighborhood_size_factor = 0.1
        self.range_size_factor = 0.1

    def __call__(self, chromosome: Chromosome):
        '''Generate a set of neighbor solutions by swapping a range of elements in the chromosome.'''
        neighbors = []
        neighborhood_size = int(len(chromosome) * self.neighborhood_size_factor)
        range_size = int(len(chromosome) * self.range_size_factor)
        for _ in range(neighborhood_size):
            neighbor = chromosome.copy()
            i = random.randint(0, len(chromosome) - range_size)
            j = random.randint(i, i + range_size)
            neighbor[i:j]['time_slot'] = neighbor[j:i]['time_slot']
            if neighbor[i]['module'] == neighbor[j]['module']:
                neighbor[i:j]['assistant'] = neighbor[j:i]['assistant']
            neighbors.append(neighbor)
        return neighbors
    
    def configure(self, neighborhood_size_factor = None, range_size_factor = None):
        self.neighborhood_size_factor = neighborhood_size_factor or self.neighborhood_size_factor
        self.range_size_factor = range_size_factor or self.range_size_factor
        return self
    
class DistanceSwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("DistanceSwapNeighborhood")
        self.distance_percentage = 0.1
        self.distance_matrix = None

    def __call__(self, chromosome: Chromosome) -> List[Chromosome]:
        '''Swap genes based on the distance between the genes'''
        neighbors = []
        
        if self.distance_matrix is None:
            self.distance_matrix = self.calculate_distance_matrix(chromosome)

        # Sort the distance from the furthest to the closest
        distance = sorted(self.distance_matrix, key=lambda distance: distance[2], reverse=True)

        # Select the top 10% of the distance
        selected_distance = distance[:int(len(distance) * self.distance_percentage)]

        # Swap the genes
        for distance in selected_distance:
            neighbor = chromosome.copy()
            self.swap_gene(neighbor, distance[0], distance[1])
            neighbors.append(neighbor)
        return neighbors
    
    def calculate_distance_matrix(self, chromosome: Chromosome) -> List[List[int]]:
        '''Calculate the distance between each gene in the chromosome'''
        distance_matrix = []
        for i in range(len(chromosome)):
            for j in range(i + 1, len(chromosome)):
                distance = self.calculate_distance(chromosome[i], chromosome[j])
                distance_matrix.append([i, j, distance])
        return distance_matrix
    
    def calculate_distance(self, gene1: GeneData, gene2: GeneData) -> int:
        '''Calculate the distance between 2 genes'''
        date_difference = abs((gene1['time_slot'].date - gene2['time_slot'].date).days)
        day_difference = abs(Constant.days.index(gene1['time_slot'].day) - Constant.days.index(gene2['time_slot'].day))
        shift_difference = abs(Constant.shifts.index(gene1['time_slot'].shift) - Constant.shifts.index(gene2['time_slot'].shift))
        return date_difference + day_difference + shift_difference
    
    def swap_gene(self, chromosome: Chromosome, index1: int, index2: int):
        '''Swap 2 genes in the chromosome'''
        chromosome[index1]['time_slot'], chromosome[index2]['time_slot'] = chromosome[index2]['time_slot'], chromosome[index1]['time_slot']
        if chromosome[index1]['module'] == chromosome[index2]['module']:
            chromosome[index1]['assistant'], chromosome[index2]['assistant'] = chromosome[index2]['assistant'], chromosome[index1]['assistant']

    def configure(self, distance_percentage = None):
        self.distance_percentage = distance_percentage or self.distance_percentage
        return self

#### Tabu List

In [31]:
# Tabu List

from collections import deque

class TabuList:
    def __init__(self, tabu_list_size: int = 10):
        self.tabu_list_size = tabu_list_size
        self.tabu_list = deque(maxlen=tabu_list_size)
    
    def __str__(self):
        return f"TabuList(tabu_list_size={self.tabu_list_size}, tabu_list={self.tabu_list})"
    
    def __contains__(self, chromosome: Chromosome):
        return chromosome in self.tabu_list
    
    def __len__(self):
        return len(self.tabu_list)
    
    def __getitem__(self, index):
        return self.tabu_list[index]
    
    def __setitem__(self, index, chromosome: Chromosome):
        self.tabu_list[index] = chromosome
    
    def __delitem__(self, index):
        del self.tabu_list[index]
    
    def __iter__(self):
        return iter(self.tabu_list)
    
    def __reversed__(self):
        return reversed(self.tabu_list)
    
    def __add__(self, chromosome: Chromosome):
        self.tabu_list.append(chromosome)
    
    # def is_tabu(self, chromosome: Chromosome) -> bool:
    #     return chromosome in self.tabu_list
    
    def configure(self, tabu_list_size: int):
        self.tabu_list_size = tabu_list_size
        self.tabu_list = deque(maxlen=tabu_list_size)
        return self

#### Strategy

##### Tabu Search

In [32]:
# Tabu Search

import time

class TabuSearch(BaseSearch):
    def __init__(self):
        super().__init__("TabuSearch")
        self.tabu_list = TabuList(10)
        self.neighborhood = RandomSwapNeighborhood()
        self.max_iteration = 1000
        self.max_time = 60
        self.max_iteration_without_improvement = 100
        self.max_time_without_improvement = 10
        self.iteration = 0
        self.time = 0
        self.iteration_without_improvement = 0
        self.time_without_improvement = 0
        self.best_chromosome = None
        self.best_fitness = None
        
        self.log = None
        self.log_detail = None

        self.debug = False

        self.repair_manager = RepairManager([RepairTimeSlot()])

    def __call__(self, chromosome: Chromosome):
        return self.run(chromosome)
    
    def run(self, chromosome: Chromosome):
        if self.debug:
            #Print the initial chromosome and configuration
            print("Search: ", self.name)
            print("Initial fitness: ", self.fitness_manager(chromosome))
            print("Neighborhood: ", self.neighborhood)
            print("Initial tabu list max size: ", self.tabu_list.max_size)
            print("Max iteration: ", self.max_iteration)
            print("Max time: ", self.max_time)
            print("Max iteration without improvement: ", self.max_iteration_without_improvement)
            print("Max time without improvement: ", self.max_time_without_improvement)
            print("--------------------------------------------------")

        # Initialize the best chromosome
        self.best_chromosome = deepcopy(chromosome)
        self.best_fitness = chromosome.fitness
        # Initialize the log
        self.log = []
        self.log_detail = []
        # Initialize the iteration
        self.iteration = 0
        self.time = 0
        self.iteration_without_improvement = 0
        self.time_without_improvement = 0
        # Start the search
        start = time.time()
        while self.iteration < self.max_iteration and self.time < self.max_time and self.iteration_without_improvement < self.max_iteration_without_improvement and self.time_without_improvement < self.max_time_without_improvement:
            #self.log.append({"iteration": self.iteration, "time": self.time, "iteration_without_improvement": self.iteration_without_improvement, "time_without_improvement": self.time_without_improvement, "fitness": self.best_fitness})
            #self.log_detail.append({"iteration": self.iteration, "time": self.time, "iteration_without_improvement": self.iteration_without_improvement, "time_without_improvement": self.time_without_improvement, "fitness": self.best_fitness, "chromosome": self.best_chromosome})
            # Get the neighbors
            neighbors = self.get_neighbors(self.best_chromosome)
            # Repair the neighbors
            # Calculate the fitness of the neighbors
            self.calculate_fitness(neighbors)
            # Select the best neighbor
            best_neighbor = self.select_best_neighbor(neighbors)
            # Check if the best neighbor is better than the current best chromosome
            if best_neighbor.fitness < self.best_fitness:
                self.best_chromosome = deepcopy(best_neighbor)
                self.best_fitness = best_neighbor.fitness
                self.iteration_without_improvement = 0
                self.time_without_improvement = 0
                self.tabu_list + best_neighbor
            else:
                self.iteration_without_improvement += 1
                self.time_without_improvement = time.time() - start - self.time
            self.iteration += 1
            self.time = time.time() - start
        return self.best_chromosome
    
    def calculate_fitness(self, neighbors: List[Chromosome]):
        '''Calculate the fitness of the neighbors'''
        for neighbor in neighbors:
            self.repair_manager(neighbor)
            neighbor.fitness = self.fitness_manager(neighbor)

    def get_neighbors(self, chromosome: Chromosome):
        '''Get the neighbors of the chromosome'''
        return self.neighborhood(chromosome)
    
    def select_best_neighbor(self, neighbors: List[Chromosome]):
        '''Select the best neighbor from the neighbors'''
        best_neighbor = neighbors[0]
        for neighbor in neighbors:
            if neighbor not in self.tabu_list and neighbor.fitness < best_neighbor.fitness:
                best_neighbor = neighbor
        return best_neighbor
    
    def configure(self, fitness_manager: FitnessManager, tabu_list: TabuList, neighborhood: BaseNeighborhood, max_iteration: int, max_time: int, max_iteration_without_improvement: int, max_time_without_improvement: int):
        '''Configure the search
        
        args:
            fitness_manager: FitnessManager
            tabu_list: TabuList
            neighborhood: BaseNeighborhood
            max_iteration: int
            max_time: int
            max_iteration_without_improvement: int
            max_time_without_improvement: int'''
        
        self.fitness_manager = fitness_manager or self.fitness_manager
        self.tabu_list = tabu_list or self.tabu_list
        self.neighborhood = neighborhood or self.neighborhood
        self.max_iteration = max_iteration or self.max_iteration
        self.max_time = max_time or self.max_time
        self.max_iteration_without_improvement = max_iteration_without_improvement or self.max_iteration_without_improvement
        self.max_time_without_improvement = max_time_without_improvement or self.max_time_without_improvement
        return self

##### Simulated Annealing

In [33]:
##### Simulated Annealing

# Simulated Annealing Class

import math
import random
import time

class SimulatedAnnealing(BaseSearch):
    def __init__(self):
        super().__init__("SimulatedAnnealing")
        self.neighborhood = RandomSwapNeighborhood()
        self.initial_temperature = 100
        self.temperature = 100
        self.temperature_threshold = 0.1
        self.cooling_rate = 0.01
        self.max_iteration = 1000
        self.iteration_without_improvement = 0
        self.max_iteration_without_improvement = 100
        self.max_time = 60
        self.best_chromosome = None
        self.best_fitness = None
        self.iteration = 0
        self.time = 0
        self.log = None
        self.log_detail = None
        self.information = None

        self.debug = False

        self.termination_reason = None

        self.repair_manager = RepairManager([RepairTimeSlot()])
        self.fitness_manager = FitnessManager([GroupAssignmentConflictFitness(), AssistantDistributionFitness()])

    def __call__(self, chromosome: Chromosome):
        return self.run(chromosome)
    
    def run(self, chromosome: Chromosome):
        if self.debug:
            #Print the initial chromosome and configuration
            print("Search: ", self.name)
            print("Initial fitness: ", self.fitness_manager(chromosome))
            print("Neighborhood: ", self.neighborhood)
            print("Initial temperature: ", self.initial_temperature)
            print("Cooling rate: ", self.cooling_rate)
            print("Max iteration: ", self.max_iteration)
            print("Max time: ", self.max_time)
            print("--------------------------------------------------")

        # Initialize the best chromosome
        self.best_chromosome = chromosome.copy()
        self.best_fitness = chromosome.fitness
        # Initialize the log
        self.log = []
        self.log_detail = []
        # Initialize the iteration
        self.iteration = 0
        self.time = 0
        # Initialize the temperature
        self.temperature = self.initial_temperature
        # Start the search
        start = time.time()
        while self.awake():
            self.log.append({"iteration": self.iteration, "time": self.time, "fitness": self.best_fitness})
            self.log_detail.append({"iteration": self.iteration, "time": self.time, "fitness": self.best_fitness, "chromosome": self.best_chromosome})
            # Get the neighbors
            neighbors = self.get_neighbors(self.best_chromosome)
            # Calculate the fitness of the neighbors
            self.calculate_fitness(neighbors)
            # Select the best neighbor
            best_neighbor = self.select_best_neighbor(neighbors)
            # Check if the best neighbor is better than the current best chromosome
            if best_neighbor.fitness < self.best_fitness:
                self.best_chromosome = best_neighbor.copy()
                self.best_fitness = best_neighbor.fitness
            else:
                # Calculate the probability of accepting the worse neighbor
                probability = self.calculate_probability(best_neighbor.fitness)
                if random.random() < probability:
                    self.best_chromosome = best_neighbor.copy()
                    self.best_fitness = best_neighbor.fitness
                    self.iteration_without_improvement = 0
                else:
                    self.iteration_without_improvement += 1
            # Cool down the temperature
            self.cool_down()
            self.iteration += 1
            self.time = time.time() - start

        # Return the best chromosome
        self.information = {"iteration": self.iteration, "time": self.time, "fitness": self.best_fitness, "chromosome": self.best_chromosome, "termination_reason": self.termination_reason}
        return self.best_chromosome
    
    def awake(self) -> bool:
        '''Check if the search should continue'''
        reached_max_iteration = self.iteration >= self.max_iteration
        reached_max_time = self.time >= self.max_time
        reached_max_iteration_without_improvement = self.iteration_without_improvement >= self.max_iteration_without_improvement
        reached_temperature_threshold = self.temperature <= self.temperature_threshold
        reached_best_fitness = self.best_fitness == 0
        self.termination_reason = "Reached max iteration" if reached_max_iteration else "Reached max time" if reached_max_time else "Reached max iteration without improvement" if reached_max_iteration_without_improvement else "Reached temperature threshold" if reached_temperature_threshold else "Reached best fitness" if reached_best_fitness else None
        return (not reached_max_iteration and not reached_max_time and not reached_max_iteration_without_improvement and not reached_temperature_threshold) and not reached_best_fitness
    
    def calculate_fitness(self, neighbors: List[Chromosome]):
        '''Calculate the fitness of the neighbors'''
        for neighbor in neighbors:
            self.repair_manager(neighbor)
            neighbor.fitness = self.fitness_manager(neighbor)

    def get_neighbors(self, chromosome: Chromosome):
        '''Get the neighbors of the chromosome'''
        return self.neighborhood(chromosome)
    
    def select_best_neighbor(self, neighbors: List[Chromosome]):
        '''Select the best neighbor from the neighbors'''
        best_neighbor = neighbors[0]
        for neighbor in neighbors:
            if neighbor.fitness < best_neighbor.fitness:
                best_neighbor = neighbor
        return best_neighbor
    
    def calculate_probability(self, neighbor_fitness: float):
        '''Calculate the probability of accepting the worse neighbor'''
        return math.exp((self.best_fitness - neighbor_fitness) / self.temperature)
    
    def cool_down(self):
        '''Cool down the temperature'''
        self.temperature *= 1 - self.cooling_rate


    def configure(self, fitness_manager: FitnessManager = None, neighborhood: BaseNeighborhood = None, initial_temperature: float = None, cooling_rate: float = None, max_iteration: int = None, max_iteration_without_improvement: int = None, max_time: int = None):
        '''Configure the search
        
        args:
            fitness_manager: FitnessManager
            neighborhood: BaseNeighborhood
            initial_temperature: float
            cooling_rate: float
            max_iteration: int
            max_time: int'''
        
        self.fitness_manager = fitness_manager or self.fitness_manager
        self.neighborhood = neighborhood or self.neighborhood
        self.initial_temperature = initial_temperature or self.initial_temperature
        self.temperature = initial_temperature or self.initial_temperature
        self.cooling_rate = cooling_rate or self.cooling_rate
        self.max_iteration = max_iteration or self.max_iteration
        self.max_time = max_time or self.max_time
        self.max_iteration_without_improvement = max_iteration_without_improvement or self.max_iteration_without_improvement

        return self
    
    def get_log(self):
        return self.log
    
    def get_log_detail(self):
        return self.log_detail


##### Test

In [34]:
settings.DEBUG = False
factory = Factory()
best_chromosome = generate_best_chromosome(2, 1)
best_chromosome.fitness

100

In [35]:
#test tabu search
group_assignment_conflict_fitness = GroupAssignmentConflictFitness()
group_assignment_conflict_fitness.configure(3, 1)

assistant_distribution_fitness = AssistantDistributionFitness()
assistant_distribution_fitness.configure(15, 50, 1, 1)

neighborhood = RandomSwapNeighborhood()
fitness_manager = FitnessManager([group_assignment_conflict_fitness, assistant_distribution_fitness])
tabu_search = TabuSearch().configure(fitness_manager, None, neighborhood, 1000, 60, 100, 10)
tabu_search.tabu_list.configure(10)

simulated_annealing = SimulatedAnnealing().configure(fitness_manager=fitness_manager,
                                                    neighborhood=neighborhood,
                                                    initial_temperature=100,
                                                    cooling_rate=0.25,
                                                    max_iteration=1000,
                                                    max_iteration_without_improvement=100,
                                                    max_time=60
                                                    )

## Hybrid Algorithm
Algoritma hibrida yang menggabungkan algoritma genetika dan tabu search. Algoritma ini berfungsi untuk memperbaiki hasil dari algoritma genetika, sekaligus mengenalkan diversity pada hasil penjadwalan agar tidak terjebak pada local optimum.

In [36]:
#Hybrid Algorithm Class
# Create a hybrid algorithm class that combine the genetic algorithm and tabu search, the genetic algorithm will be used to generate the initial solution and focus on the exploration, while the tabu search will be used to improve the solution and focus on the exploitation.

class HybridAlgorithm(GeneticAlgorithm):
    def __init__(self):
        super().__init__()
        self.local_search = TabuSearch()
    
    def __str__(self):
        return f"HybridAlgorithm(factory={self.factory}, population_size={self.population_size}, selection_manager={self.selection_manager}, crossover_manager={self.crossover_manager}, mutation_manager={self.mutation_manager}, repair_manager={self.repair_manager}, elitism_size={self.elitism_size}, local_search={self.local_search})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, max_iteration):
        return self.run(max_iteration)
    
    def run(self, max_iteration: int):
        # Initialize the population
        population = self._init_population()
        # Calculate the fitness of the population
        population.calculate_fitness()
        # Sort the population based on fitness
        population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
        # Initialize the best chromosome
        best_chromosome = deepcopy(population[0])
        # Initialize the iteration
        iteration = 0
        # Start the hybrid algorithm
        while iteration < max_iteration and best_chromosome.fitness > 0:
            # Evolve the population, crossover and mutation happens inside this function
            population, elitism = self._evolve_population(population)

            # Add the elitism and the local search to the population
            population.add_chromosome(elitism)
            population.calculate_fitness()

            # Introduction of local search, for possible improvement of previous best chromosome
            local_search = self.local_search(best_chromosome)
            population.add_chromosome(local_search)

            # Sort the population based on fitness
            population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
            #remove the worst chromosome, so the population size is still the same
            population.pop()
            # Check if the best chromosome is better than the current best chromosome
            if population[0].fitness < best_chromosome.fitness:
                best_chromosome = deepcopy(population[0])

            iteration += 1
        return best_chromosome
    
    def configure(self, population_size: int = 10, elitism_size: int = 1, factory: Factory = None, fitness_manager: FitnessManager = None, selection_manager: SelectionManager = None, crossover_manager: CrossoverManager = None, mutation_manager: MutationManager = None, repair_manager: RepairManager = None, elitism_selection: ElitismSelection = None, local_search: TabuSearch = None):
        '''Configure the hybrid algorithm, use None to use the default value.
        '''
        self.population_size = population_size or self.population_size
        self.elitism_size = elitism_size or self.elitism_size
        self.elitism_selection = elitism_selection or self.elitism_selection
        self.factory = factory or self.factory
        self.fitness_manager = fitness_manager or self.fitness_manager
        self.selection_manager = selection_manager or self.selection_manager
        self.crossover_manager = crossover_manager or self.crossover_manager
        self.mutation_manager = mutation_manager or self.mutation_manager
        self.repair_manager = repair_manager or self.repair_manager
        self.local_search = local_search or self.local_search
        return self


In [41]:
#test hybrid algorithm

settings.DEBUG = False

#factory
factory = Factory()

#fitness
group_assignment_conflict_fitness = GroupAssignmentConflictFitness().configure(3, 1)
assistant_distribution_fitness = AssistantDistributionFitness().configure(15, 50, 1, 1)
fitness_manager = FitnessManager([group_assignment_conflict_fitness, assistant_distribution_fitness])

#selection
selection_manager = SelectionManager([RouletteWheelSelection(), TournamentSelection(), ElitismSelection()])

#Crossover
crossover_manager = CrossoverManager([SinglePointCrossover(), TwoPointCrossover(), UniformCrossover()])

#Mutation
mutation_manager = MutationManager([SwapMutation(), ShiftMutation(), RandomMutation()])

#Repair
repair_manager = RepairManager([RepairTimeSlot()])

#Elitism
elitism_size = 2
elitism_selection = ElitismSelection()

#Tabu Search
neighborhood = RandomSwapNeighborhood()
tabu_list = TabuList(50)
tabu_search = TabuSearch().configure(fitness_manager = fitness_manager,
                                    tabu_list = tabu_list,
                                    neighborhood = neighborhood,
                                    max_iteration = 1000,
                                    max_time = 60,
                                    max_iteration_without_improvement = 100,
                                    max_time_without_improvement = 5)

simulated_annealing = SimulatedAnnealing().configure(fitness_manager=fitness_manager,
                                                    neighborhood=neighborhood,
                                                    initial_temperature=100,
                                                    cooling_rate=0.1,
                                                    max_iteration=1000,
                                                    max_iteration_without_improvement=100,
                                                    max_time=60
                                                    )

hybrid_algorithm = HybridAlgorithm().configure(population_size = 10, 
                            elitism_size = elitism_size,
                            factory = factory, 
                            fitness_manager = fitness_manager, 
                            selection_manager = selection_manager, 
                            crossover_manager = crossover_manager,
                            mutation_manager = mutation_manager, 
                            repair_manager = repair_manager, 
                            elitism_selection = elitism_selection,
                            local_search = tabu_search)

genetic_algorithm = GeneticAlgorithm().configure(population_size = 25,
                            elitism_size = elitism_size, 
                            factory = factory, 
                            fitness_manager = fitness_manager, 
                            selection_manager = selection_manager, 
                            crossover_manager = crossover_manager,
                            mutation_manager = mutation_manager, 
                            repair_manager = repair_manager, 
                            elitism_selection = elitism_selection)

In [42]:
new_chromosome = hybrid_algorithm.run(5)

In [44]:
new_chromosome.fitness

9

In [271]:
chromosome_genetic = genetic_algorithm.run(500)

In [41]:
chromosome_tabu = tabu_search.run(chromosome_genetic)

In [272]:
chromosome_genetic.fitness

31